In [1]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from scipy.io import loadmat
import os
from glob import glob

In [2]:
# REPLACE WITH THE PATH TO YOUR DOWNLOAD OF OFC-2 #
dpath = '/Users/alex/Dropbox/data/crcns/ofc-2/'

In [3]:
rats = os.listdir(dpath)
sessions = {r: os.listdir(os.path.join(dpath, r)) for r in rats}
tmax = 6.0  # maximum time for a trial

def load(rat=0, sess=0):
    r = rats[rat] if type(rat)==int else rat
    p = os.path.join(dpath, r, sessions[r][sess])
    spiketimes = [loadmat(f)['TS'].ravel()/1e4 for f in glob(os.path.join(p, 'Sc*'))]
    
    ev1 = loadmat(os.path.join(p, 'TrialEvents.mat'))
    ev2 = loadmat(os.path.join(p, 'TrialEvents2.mat'))
    
    # collect the important metadata
    info = {
        'correct': ev2['Correct'].ravel(),
        'stim': ev2['OdorCategory'].ravel(),
        'choice': ev2['ChoiceDir'].ravel(),
        'ratio': ev2['OdorRatio'].ravel(),
        'odor_in': ev1['TrialStart'].ravel() + ev1['OdorPokeIn'].ravel(),
        'odor_out': ev1['TrialStart'].ravel() + ev1['OdorPokeOut'].ravel(),
        'water_in': ev1['TrialStart'].ravel() + ev1['WaterPokeIn'].ravel(),
        'water_on': ev1['TrialStart'].ravel() + ev2['WaterValveOn'].ravel(),
        'water_out': ev1['TrialStart'].ravel() + ev1['WaterPokeOut'].ravel(),
    }
    
    # align relative to odor_in
    for k in ('odor_out', 'water_in', 'water_on', 'water_out'):
        info[k] = (info[k] - info['odor_in']) / tmax

    # throw out incorrect trials
    idx = np.isfinite(info['water_out'])
    info = {k: v[idx] for k, v in info.items()}
    
    # throw out spikes preceding first trial
    spiketimes = np.asarray([s[s>info['odor_in'][0]] for s in spiketimes])
    
    trials, binned = [], []
    times, neurons = [], []
    for n, st in enumerate(spiketimes):
        tr, M = bin_neuron(st, info)
        trials += tr.tolist()
        binned.append(M)
        times += st.tolist()
        neurons += [n for _ in range(len(st))]

    trials, times, neurons, binned = np.array(trials), np.array(times), np.array(neurons), np.array(binned)
    idx = trials >= 0
    binned = binned.transpose(1, 2, 0)
    for k in info.keys():
        info[k] = info[k][np.unique(trials[idx])]
    info.pop('odor_in')  # this is aligned to be timepoint zero, so it is redundant/confusing to leave it in.
    return trials[idx], times[idx]/tmax, neurons[idx], binned[np.unique(trials[idx])], info

In [4]:
from numba import jit

@jit(nopython=True)
def _bin(trials, times, binned, start_times):
    # trial counter
    K = len(start_times)
    k = 0

    # number of spike bins in a trial
    nbins = binned.shape[1]
    
    # add each spike to M
    for i in range(len(times)):
        # get spike time
        s = times[i]
        # ignore spikes preceding first trial
        if s < start_times[k]:
            continue
        # advance to next trial
        while (k + 1 < K) and (s > start_times[k+1]):
            k += 1
        # find bin
        t = int(nbins * ((s-start_times[k]) / tmax))
        if t < nbins:
            binned[k, t] += 1
            trials[i] = k
            times[i] = times[i] - start_times[k]

def bin_neuron(st, info):
    K = len(info['odor_out'])
    M = np.zeros((K, 200))
    trials = np.full(len(st), -1)
    _bin(trials, st, M, info['odor_in'])
    return trials, M

In [5]:
fields = ['trials', 'times', 'neurons', 'binned', 'metadata']
data = {k: v for k, v in zip(fields, load(rat=0, sess=3))}

In [6]:
import deepdish as dd
dd.io.save('ofc2_data.h5', data)